In [4]:
import pandas as pd
import numpy as np
import pickle
import os
import json
import collections
import time


from typing import List, Dict, NoReturn, Union, Any, Optional, Tuple
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report,accuracy_score, balanced_accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTENC, ADASYN, RandomOverSampler
from sklearn.utils import resample

In [5]:
df_one_hot_vectors = pd.read_csv('/home/marcin/PycharmProjects/Engineering_Thesis/dataset/CriteoSearchData-sorted-no-duplicates.csv',
             sep='\t',
             nrows=100,
             skiprows=range(1,50000),
             low_memory=True)

In [6]:
df_one_hot_vectors[:3]

,Sale,SalesAmountInEuro,time_delay_for_conversion,click_timestamp,nb_clicks_1week,product_price,product_age_group,device_type,audience_id,product_gender,...,product_category(3),product_category(4),product_category(5),product_category(6),product_category(7),product_country,product_id,product_title,partner_id,user_id
0,0,-1.00,-1,1596478734,10,0.00,-1,D7D1FB49049702BF6338894757E0D959,-1,-1,...,9A45982A2EC0EA28F61BF772AE7B1298,AD527420C750BB93826A1269E32C1C0A,-1,-1,-1,57A1D462A03BD076E029CF9310C11FC5,91EF214D4E3CC30B7CD09F50A3121ABE,24D4F8D30C3029FD1D24B305FDC06DBE E6788D2ECB900...,E3DDEB04F8AFF944B11943BB57D2F620,2C3E1BCDD2375038B669E32DA2726742
1,0,-1.00,-1,1596478736,-1,0.00,-1,D7D1FB49049702BF6338894757E0D959,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,AA00F41CFEFBA02962819036DD6CA017,A28439FBB72472309F7AF16C2D5D1192
2,1,55.62,204,1596478736,-1,41.13,-1,7E56C27BFF0305E788DA55A029EC4988,-1,-1,...,-1,-1,-1,-1,-1,57A1D462A03BD076E029CF9310C11FC5,7260EF49A39013E419F0841760419D6F,-1,E3DDEB04F8AFF944B11943BB57D2F620,F145C3DE8AFF6317850E4E31FC6EECC3


In [7]:

df_one_hot_vectors = df_one_hot_vectors.dropna(axis=0) # usuwanie wierszy, które zawierają null
df_one_hot_vectors.isnull().sum()

Sale                         0
SalesAmountInEuro            0
time_delay_for_conversion    0
click_timestamp              0
nb_clicks_1week              0
product_price                0
product_age_group            0
device_type                  0
audience_id                  0
product_gender               0
product_brand                0
product_category(1)          0
product_category(2)          0
product_category(3)          0
product_category(4)          0
product_category(5)          0
product_category(6)          0
product_category(7)          0
product_country              0
product_id                   0
product_title                0
partner_id                   0
user_id                      0
dtype: int64

In [16]:
X = df_one_hot_vectors.iloc[:, 3:].values
y = df_one_hot_vectors.iloc[:, :1].values.ravel()  # tutaj powinny być chyba 3 kolumny
X
# class_weights = compute_class_weight('balanced',
#                      np.unique(y),
#                      y)
# class_weight_dictionary = {0:class_weights[0], 1:class_weights[1]}

# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

# pca = PCA(n_components=500, random_state=1)
# x_train = pca.fit_transform(x_train)
# x_test = pca.transform(x_test)


# start = time.time()
# smote_nc = SMOTENC(categorical_features=list(np.arange(0,96)),random_state=1, n_jobs=-1) 
# x_train, y_train = smote_nc.fit_resample(x_train, y_train)
# end = time.time()
# print(end - start)


# adasyn = ADASYN(random_state=1)
# x_train, y_train = adasyn.fit_resample(x_train, y_train)

# ros = RandomOverSampler(random_state=1)
# x_train, y_train = ros.fit_sample(x_train, y_train)
    
# x_train = np.vstack((x_train[y_train==0], x_train))
# y_train = np.hstack((y_train[y_train==0], y_train))
# x_sampled, y_sampled = resample(X[y==1],
#                                y[y==1],
#                                replace=True,
#                                n_samples =X[y==0].shape[0],
#                                random_state=1)

array([[1596478734, 10, 0.0, ...,
        '24D4F8D30C3029FD1D24B305FDC06DBE E6788D2ECB900F4665CC3BE6E77E1111 B48D3166F24B324D888F3D96517A6253',
        'E3DDEB04F8AFF944B11943BB57D2F620',
        '2C3E1BCDD2375038B669E32DA2726742'],
       [1596478736, -1, 0.0, ..., '-1',
        'AA00F41CFEFBA02962819036DD6CA017',
        'A28439FBB72472309F7AF16C2D5D1192'],
       [1596478736, -1, 41.13, ..., '-1',
        'E3DDEB04F8AFF944B11943BB57D2F620',
        'F145C3DE8AFF6317850E4E31FC6EECC3'],
       ...,
       [1596478785, -1, 0.0, ..., '-1',
        '9946AE215D24E4E0CD9268ECB054EBC8',
        '7D3C7B8CA0DB32B38B695DD934E9CA9B'],
       [1596478785, 29, 0.0, ...,
        '6AE36582B79A724234F5661FA6BF2DBB 5BDBD987318370EF6D856D3B90405B0F DF2CEAA8C60F7F00D24401EA9CAB5731 79C4C045B84F5C51D8EE9D7B5B8819A2 2E6B9CE0EBD219441DEE5AFD30F2AC08 5D1EF27EF97B13C18BB34263AABA8BBC',
        '0225D81C97298F6FD63C8868EE8E2228',
        '3FFCE49BFA82298F238564E1C71D373E'],
       [1596478785, 29, 15.0, ...,

In [17]:
print('Liczba etykiet w zbiorze y:', np.bincount(y))
print('Liczba etykiet w zbiorze y_train:', np.bincount(y_train))
print('Liczba etykiet w zbiorze y_test:', np.bincount(y_test))

# lr = make_pipeline(StandardScaler(),
# #                        PCA(n_components=100),
#                         SGDClassifier(loss='log', verbose=0, n_jobs=-1, random_state=1, tol=1e-3, max_iter=1000, class_weight='balanced'))

# lr = SGDClassifier(loss='log', verbose=0, n_jobs=-1, random_state=1, tol=1e-3, max_iter=1000, penalty='l2')#, class_weight=class_weight_dictionary)
lr = SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000, penalty='l1', alpha=1e-05)

sc = StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_train_std = normalize(x_train_std, norm='l2')
x_test_std = sc.transform(x_test)
x_test_std = normalize(x_test_std, norm='l2')

Liczba etykiet w zbiorze y: [42756  7244]
Liczba etykiet w zbiorze y_train: [34205 34198]
Liczba etykiet w zbiorze y_test: [8551 1449]


In [18]:
# start = time.time()
# scores = cross_val_score(estimator=lr,
#                         X=x_train_std,
#                         y=y_train,
#                         cv=10,
#                         n_jobs=1)
# end = time.time()
# print(end-start)
# print(scores)
# print('Dokładność sprawdzianu: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

start = time.time()
lr.fit(x_train_std, y_train)
end = time.time()
print("Czas treningu: {0}".format((end-start)))
print("LOG: creating model DONE")

print(collections.Counter(y_test))
y_pred = lr.predict(x_test_std)
print(collections.Counter(y_pred))

print("LOG: testing model DONE")
print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))

print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())

print('classification_report :\n',classification_report(y_test,y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

Czas treningu: 5.189800024032593
LOG: creating model DONE
Counter({0: 8551, 1: 1449})
Counter({0: 8752, 1: 1248})
LOG: testing model DONE
balanced_accuracy_score: 0.6973967101462996
accuracy_score: 0.8643
Nieprawidłowo sklasyfikowane próbki: 1357
classification_report :
               precision    recall  f1-score   support

           0       0.91      0.93      0.92      8551
           1       0.54      0.46      0.50      1449

    accuracy                           0.86     10000
   macro avg       0.72      0.70      0.71     10000
weighted avg       0.86      0.86      0.86     10000

[[7973  578]
 [ 779  670]]


In [ ]:
from sklearn.model_selection import validation_curve, learning_curve
import matplotlib.pyplot as plt

param_range = [1e-15,1e-14,1e-13,1e-12,1e-11,1e-10,1e-9,1e-8,1e-7]
train_scores, test_scores = validation_curve(
                estimator=lr, 
                X=x_train_std, 
                y=y_train, 
                param_name='alpha', 
                param_range=param_range,
                cv=10)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean, 
         color='blue', marker='o', 
         markersize=5, label='Dokładność uczenia')

plt.fill_between(param_range, train_mean + train_std,
                 train_mean - train_std, alpha=0.15,
                 color='blue')

plt.plot(param_range, test_mean, 
         color='green', linestyle='--', 
         marker='s', markersize=5, 
         label='Dokładność walidacji')

plt.fill_between(param_range, 
                 test_mean + test_std,
                 test_mean - test_std, 
                 alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parametr C')
plt.ylabel('Dokładność')
plt.ylim([0.4, 0.8])
plt.tight_layout()
#plt.savefig('rysunki/06_06.png', dpi=300)
plt.show()

In [ ]:
plt.plot(param_range, train_mean, 
         color='blue', marker='o', 
         markersize=5, label='Dokładność uczenia')

plt.fill_between(param_range, train_mean + train_std,
                 train_mean - train_std, alpha=0.15,
                 color='blue')

plt.plot(param_range, test_mean, 
         color='green', linestyle='--', 
         marker='s', markersize=5, 
         label='Dokładność walidacji')

plt.fill_between(param_range, 
                 test_mean + test_std,
                 test_mean - test_std, 
                 alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parametr C')
plt.ylabel('Dokładność')
plt.ylim([0.4, 0.8])
plt.tight_layout()
#plt.savefig('rysunki/06_06.png', dpi=300)
plt.show()

In [ ]:
# compute_class_weight('balanced', np.unique(y), y)

In [ ]:
# save the model to disk
pickle.dump(lr, open('model.sav', 'wb'))
pickle.dump(sc, open('standard_scaler.sav', 'wb'))

In [ ]:
# load the model from disk
lr = pickle.load(open('model.sav', 'rb'))
sc = pickle.load(open('standard_scaler.sav', 'rb'))

In [ ]:
df_one_hot_vectors = pd.read_csv('dataset_transformed_1mln_REDUCED.csv',
             sep='\t',
             nrows=50000,
             skiprows=range(1,150000),
             header=0,
             low_memory=True)

In [ ]:
x = df_one_hot_vectors.iloc[:, 3:].values
y = df_one_hot_vectors['Sale'].values.ravel()

x_sampled, y_sampled = resample(x[y==1],
                               y[y==1],
                               replace=True,
                               n_samples =x[y==0].shape[0],
                               random_state=1)

x = np.vstack((x[y==0], x_sampled))
y = np.hstack((y[y==0], y_sampled))


x = sc.transform(x)
x = normalize(x, norm='l2')
y = np.array([int(i) for i in y])
lr.partial_fit(x, y, classes=np.array([0, 1]))

In [ ]:
# save the model to disk
pickle.dump(lr, open('model2.sav', 'wb'))
pickle.dump(sc, open('standard_scaler2.sav', 'wb'))

In [ ]:
# load the model from disk
lr = pickle.load(open('model2.sav', 'rb'))
sc = pickle.load(open('standard_scaler2.sav', 'rb'))

In [ ]:
df_one_hot_vectors_test = pd.read_csv('dataset_transformed_1mln_REDUCED.csv',
             sep='\t',
             nrows=50000,
#              skiprows=range(1,50000),
             low_memory=True)

In [ ]:
x_test = df_one_hot_vectors_test.iloc[:, 3:].values
y_test = df_one_hot_vectors_test.iloc[:, :1].values.ravel()  # tutaj powinny być chyba 3 kolumny

x_test_std = sc.transform(x_test)
x_test_std = normalize(x_test_std, norm='l2')

print(collections.Counter(y_test))
y_pred = lr.predict(x_test_std)
print(collections.Counter(y_pred))

In [ ]:
print(accuracy_score(y_test, y_pred))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('Dokładność: %.2f' % lr.score(x_test_std, y_test))

In [ ]:
print('Liczba etykiet w zbiorze y:', np.bincount(y_test))

In [ ]:
# print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('Dokładność: %.2f' % lr.score(x_test_std, y_test))

print('accuracy = ',lr.score(x_test_std, y_test).round(2),
      'precision = ',precision_score(y_test, y_pred).round(2), 
      'recall = ',recall_score(y_test, y_pred).round(2), 
      'f1_score = ',f1_score(y_test, y_pred).round(2))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

In [20]:
df_one_hot_vectors_test = pd.read_csv('dataset_transformed_1mln_REDUCED.csv',
             sep='\t',
             nrows=50000,
#                  skiprows=range(1,250000),
             low_memory=True)
    
for i in range(20):
    # PREDYKCJA
    print('Iteracja: {0}'.format(i))

    x_test = df_one_hot_vectors_test.iloc[:, 3:].values
    y_test = df_one_hot_vectors_test.iloc[:, :1].values.ravel()  # tutaj powinny być chyba 3 kolumny
    
    x_test = pca.transform(x_test)
    x_test_std = sc.transform(x_test)
    x_test_std = normalize(x_test_std, norm='l2')

    print(collections.Counter(y_test))
    y_pred = lr.predict(x_test_std)
    print(collections.Counter(y_pred))

    print(accuracy_score(y_test, y_pred))
    print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
    print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
    print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
    print('classification_report :\n',classification_report(y_test,y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print(confmat)
    print('\n')
    
    # NAUKA
    df_one_hot_vectors = pd.read_csv('dataset_transformed_1mln_REDUCED.csv',
                 sep='\t',
                 nrows=50000,
                 skiprows=range(1,100000+50000*i),
                 header=0,
                 low_memory=True)

    
    df_one_hot_vectors = df_one_hot_vectors.dropna(axis=0) # usuwanie wierszy, które zawierają null

    x = df_one_hot_vectors.iloc[:, 3:].values
    y = df_one_hot_vectors['Sale'].values.ravel()
    
    x = pca.transform(x)


#     smote_nc = SMOTENC(categorical_features=list(np.arange(0,4225)), random_state=1)
#     x, y = smote_nc.fit_resample(x, y)

#     adasyn = ADASYN(random_state=1)
    x, y = adasyn.fit_resample(x, y)

#     ros = RandomOverSampler(random_state=1)
#     x, y = ros.fit_sample(x, y)

#     x_sampled, y_sampled = resample(x[y==1],
#                                    y[y==1],
#                                    replace=True,
#                                    n_samples =x[y==0].shape[0],
#                                    random_state=1)

#     x = np.vstack((x[y==0], x_sampled))
#     y = np.hstack((y[y==0], y_sampled))


    x = sc.transform(x)
    x = normalize(x, norm='l2')
    y = np.array([int(i) for i in y])
    lr = lr.partial_fit(x, y, classes=np.array([0, 1]))

Iteracja: 0
Counter({0: 44126, 1: 5874})
Counter({0: 44368, 1: 5632})
0.87692
Nieprawidłowo sklasyfikowane próbki: 6154
balanced_accuracy_score: 0.6942884608876327
accuracy_score: 0.87692
classification_report :
               precision    recall  f1-score   support

           0       0.93      0.93      0.93     44126
           1       0.48      0.46      0.47      5874

    accuracy                           0.88     50000
   macro avg       0.70      0.69      0.70     50000
weighted avg       0.87      0.88      0.88     50000

[[41170  2956]
 [ 3198  2676]]


Iteracja: 1
Counter({0: 44126, 1: 5874})
Counter({0: 44908, 1: 5092})
0.88196
Nieprawidłowo sklasyfikowane próbki: 5902
balanced_accuracy_score: 0.6865182058046516
accuracy_score: 0.88196
classification_report :
               precision    recall  f1-score   support

           0       0.93      0.94      0.93     44126
           1       0.50      0.43      0.46      5874

    accuracy                           0.88     50

[[40266  3860]
 [ 2980  2894]]


Iteracja: 15
Counter({0: 44126, 1: 5874})
Counter({0: 43075, 1: 6925})
0.86254
Nieprawidłowo sklasyfikowane próbki: 6873
balanced_accuracy_score: 0.7073189759581435
accuracy_score: 0.86254
classification_report :
               precision    recall  f1-score   support

           0       0.93      0.91      0.92     44126
           1       0.43      0.50      0.46      5874

    accuracy                           0.86     50000
   macro avg       0.68      0.71      0.69     50000
weighted avg       0.87      0.86      0.87     50000

[[40164  3962]
 [ 2911  2963]]


Iteracja: 16
Counter({0: 44126, 1: 5874})
Counter({0: 44876, 1: 5124})
0.88968
Nieprawidłowo sklasyfikowane próbki: 5516
balanced_accuracy_score: 0.7063140882461653
accuracy_score: 0.88968
classification_report :
               precision    recall  f1-score   support

           0       0.93      0.95      0.94     44126
           1       0.53      0.47      0.50      5874

    accuracy   

In [ ]:
start = time.time()
adasyn = ADASYN(random_state=1)
x_train, y_train = adasyn.fit_resample(x_train, y_train)
end = time.time()
print(end-start)

In [ ]:
from sklearn.model_selection import GridSearchCV

start = time.time()
pipe = make_pipeline(StandardScaler(),
#                      PCA(random_state=1),
#                      ADASYN(random_state=1),
                     SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000))

param_grid = [{'sgdclassifier__alpha': [1e-15, 1e-10, 1e-5, 0.01], 
               'sgdclassifier__penalty': ['l1', 'l2']
              }]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring= 'balanced_accuracy', 
                  cv=10,
                  n_jobs=-1)
gs = gs.fit(x_train, y_train)
print(gs.best_score_)
print(gs.best_params_)
end = time.time()
print(end-start)

In [ ]:
print(gs.)